#### This program will function as an LLM Tutor using Ollama Locally

Import Statements

In [40]:
import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display

Connecting to Ollama Locally

In [41]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

#### Setting up the Environment

System Prompt

In [42]:
# The system prompt is what explains the context of the situation to the frontier model
# It tells them what kind of task they are performing and what tone to use

system_prompt = "You are provided with a question.. \
    You are going to best answer the question as \
    if you are a tutor of the relevant subject matter.\n"
system_prompt += "You should respond with normal conversational prose"

User Prompt

In [43]:
# A function that writes a User Prompt that asks for summaries of websites:
# The user prompt is the actual conversation itself
# The converstaion start and the role of the LLM is to figure out what way to respond to the user prompt in the context of the system prompt

def user_prompt_for(question):
    user_prompt = f"You are looking at a question {question}"
    user_prompt += "\nThis question is about how to understand a piece of code; \
please provide a walkthrough on what this code is doing.\n\n"
    user_prompt += question
    return user_prompt

Define Answer for Question Function

In [44]:
# Create a messages list using the same format that we used for OpenAI

def answer_for_question(question):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(question)}
    ]

Define Question List

In [45]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

Answer Question Function

In [46]:
def answer_question(question):
    # Create the question object
    question = question
    
    # Generate the messages payload
    messages = answer_for_question(question)
    
    # Create the payload for the Ollama API
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False  # Disable streaming for simplicity
    }
    
    # Make the request to the Ollama API
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    
    # Extract the summary from the response
    if response.status_code == 200:
        return response.json().get('message', {}).get('content', "No content in response.")
    else:
        return f"Error: {response.status_code} - {response.text}"


Displaying Answer Function

In [47]:
# A function to display this nicely in the Jupyter output, using markdown

def display_answer(question):
    # Generate the answer using the appropriate function
    answer = answer_question(question)
    # Display the answer as Markdown
    display(Markdown(answer))


In [48]:
display_answer(question)

Let's break down this line of code together.

This code snippet appears to be part of a Python context manager or a loop, possibly within a list comprehension. It uses the `yield from` syntax, which is a feature introduced in Python 3.3 as part of the generators.

Here's what this code does:

1. **List Comprehension**: The expression inside the `{}` is a list comprehension. A list comprehension is a concise way to create a new list by performing an operation on each item in an existing list or other iterable.

2. **Filtering with `if` condition**: The `for book in books if book.get("author")` part filters out books that don't have an "author" key.

3. **Yield from**: The `yield from` statement is used to delegate to a subiterator, which in this case is the generator expression `{book.get("author") for book in books if book.get("author")}`.

So, what does it do?

It yields the authors of all books that have an author specified. In other words, it iterates over each book in the `books` list (or iterable), and if a book has an "author" key, it yields the value of that key.

Here's why:

The purpose of using `yield from` here is to avoid having to manually iterate over the subgenerator expression. If we were to do that, we would have to use another loop or recursion, which could be more complex and less efficient.

By using `yield from`, we're essentially saying: "Take this list comprehension expression as a subiterator, and yield its values one by one, without having to handle the iteration logic myself."

To illustrate this with a simple example:

Let's say you have a dictionary where each key is a book title, and its corresponding value is another dictionary containing the author. Here's how this would look like:

```python
books = [
    {"title": "Book 1", "author": "Author 1"},
    {"title": "Book 2", "author": None}, # No author for some books
    {"title": "Book 3", "author": "Author 3"}
]
```

Now, if we use this code:

```python
for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```

It will output: `Author 1`, `Author 3`.